# ptyLab simulation tutorial
At this point it is assumed that ptyLab has already been successfully installed. In this short tutorial we will simulate a conventional ptychography experiment. To simulate a ptychography experiment the following steps have to be performed.

1. The geometry of the experiment needs to be specified. The following variables need to be defined.
    - Wavelength of the illuminating beam
    - Sample to detector distance (z0)
    - Detector pixel number (Nd)
    - Detector pixel size (dxd)
2. The illuminating beam (which we refer to as probe) has to be defined 
3. The object to be simulated needs to be defined
4. The scan grid, i.e. the coordinates with which the object is scanned, must be generated.
5. For each position a diffraction pattern needs to be calculated.

Finally, at the end of this tutorial, the object is reconstructed from the created diffraction images.

## 1. Geometry of the experiment

First we need to import all necessary packages.

In [1]:
# this makes the plots inline, if you want the plot windows out, uncomment the two lines below
# %matplotlib notebook 
import matplotlib
matplotlib.use('tkagg')

# First all required modules must be imported
import numpy as np
from fracPy.utils.utils import circ, gaussian2D, cart2pol
from fracPy.utils.scanGrids import GenerateNonUniformFermat
from fracPy.operators.operators import aspw
from fracPy.utils.visualisation import hsvplot, show3Dslider
import matplotlib.pylab as plt
from scipy.signal import convolve2d
import os
import h5py


Wrong skimage version installed
Cupy not available, will not be able to run GPU based computation
Cupy not installed
Cupy not available, will not be able to run GPU based computation
Cupy not available, will not be able to run GPU based computation
Cupy not available, will not be able to run GPU based computation
Cupy not available, will not be able to run GPU based computation
Cupy not available, will not be able to run GPU based computation
Cupy not available, will not be able to run GPU based computation
Cupy not available, will not be able to run GPU based computation
Cupy not available, will not be able to run GPU based computation


In the next step we define the geometry of the experiment and calculate the grids in the sample and detector plane. The geometry of the experiment for a standard conventional ptychography experiment is defined by three parameter:

1. The wavelength of the illuminating beam, which we assume here to be 632.8 nm
2. The distance between the sample and the detector (which we name zo)
3. The shape (Nd) and pixelsize (dxd) of the detector

In [2]:
# Set the geometry of the Experiment
wavelength = 632.8e-9
zo = 5e-2
Nd = 2**7
dxd = 16 * 4.5e-6

# Calculate the length of the detector
Ld = Nd * dxd

# probe coordinates
dxp = wavelength * zo / Ld
Np = Nd
Lp = dxp*Np
xp = np.arange(-Np//2,Np//2)*dxp
Xp,Yp = np.meshgrid(xp,xp)

# object coordinates
No = 2**9
dxo = dxp
Lo = dxo * No
xo = np.arange(-No//2,No//2)*dxo
Xo,Yo = np.meshgrid(xo,xo)


## 2. Definition of the illuminating beam
In the next step we define the beam (probe), which we want to use for our simulation. Here, a pinhole is simulated which is convolved with a gaussian kernel.

In [3]:
# generate illumination
# note: simulate focused beam
# goal: 1:1 image iris through (low-NA) lens with focal length f onto an object
f = 5e-3 # focal length of lens, creating a focused probe
pinhole = circ(Xp, Yp, Lp/2)
pinhole = convolve2d(pinhole, gaussian2D(5, 1).astype(np.float32), mode='same')


Next, we propagate probe on the sample using a propagator and use the matplotlib to plot the probe.

In [4]:
# propagate to lens
probe = aspw(pinhole, 2 * f, wavelength, Lp)[0]

# multiply with quadratic phase and aperture
aperture = circ(Xp, Yp, 3*Lp/4)
aperture = convolve2d(aperture, gaussian2D(5, 3).astype(np.float32), mode='same')
probe = probe * np.exp(-1.j*2*np.pi/wavelength*(Xp**2+Yp**2)/(2*f)) * aperture
probe = aspw(probe, 2*f, wavelength, Lp)[0]

plt.figure(figsize=(5,5), num=1)
ax1 = plt.subplot(121)
hsvplot(probe, ax=ax1, pixelSize=dxp)
ax1.set_title('complex probe')
plt.subplot(122)
plt.imshow(abs(probe)**2)
plt.title('probe intensity')
plt.show(block=False)

## 3. Definition of an object
In this example we define a binary spiral as a test object. The object needs to have a larger array size than the probe, since the probe is scanned over the object.

In [5]:
# generate object
d = 1e-3   # the smaller this parameter the larger the spatial frequencies in the simulated object
b = 33     # topological charge (feel free to play with this number)
theta, rho = cart2pol(Xo, Yo)
t = (1 + np.sign(np.sin(b * theta + 2*np.pi * (rho/d)**2)))/2
t = t*circ(Xo, Yo, Lo)*(1-circ(Xo, Yo, 200*dxo))\
    +circ(Xo, Yo, 130*dxo)
obj = convolve2d(t, gaussian2D(5, 3), mode='same')  # smooth edges

plt.figure(figsize=(5,5))
ax = plt.axes()
hsvplot(np.squeeze(obj), ax=ax)
ax.set_title('complex Object')
plt.show(block=False)

## 4. Definition of the scan map
In the next step we create a scan grid with 100 points. For this purpose, we use a ptyLab internal function that generates a grid in which the scan points are distributed as homogeneously as possible (Fermat's spiral).

In [6]:
numPoints = 100   # number of points
radius = 100    # radius of final scan grid (in pixels)
p = 1    # p = 1 is standard Fermat;  p > 1 yields more points towards the center of grid
y_coord, x_coord = GenerateNonUniformFermat(numPoints, radius=radius, power=p)

# show scan grid
plt.figure(figsize=(5, 5))
plt.plot(y_coord, x_coord, 'o')
plt.title('scan grid')
plt.show(block=False)

In the next step we calculate the physical coordinates (in meters) and save them together in a single array, which we call encoder. The scan positions are plotted for illustration on the simulated object.

In [7]:
# Calculate 
encoder = np.vstack((y_coord*dxo, x_coord*dxo)).T

# prevent negative indices by centering spiral coordinates on object
positions = np.round(encoder/dxo)
offset = 50
positions = (positions+No//2-Np//2+offset).astype(int)

# get number of positions
numFrames = len(x_coord)

# show scan grid on object
plt.figure(figsize=(5,5))
ax1 = plt.axes()
hsvplot(np.squeeze(obj), ax=ax1)
ax1.plot(positions[:,1] + Np//2, positions[:,0] + Np//2, '.', c="red")
ax1.set_title('Scan grid on sample')
plt.show(block=False)

## 5. Calculate diffraction data
Finally we have all the information to calculate the diffraction patterns. For this purpose we create a 3-dimensionale array, which is used to store for each position the diffraction data. We name this array ptychogram. 

In [10]:
# generate the ptychogram array
ptychogram = np.zeros((numFrames, Nd, Nd))

In the next step, we iterate through all the calculated positions and slice the object according to the size of the probe-array. The slice is named object patch. Since the object patch has the same size as the sample, the exit-surface-wave (esw) can be calculated in the next step. The esw represents the electric field directly behind the sample. Thus, the electric field in the detector plane can be calculated by the Fourier transform of the esw. Here it is assumed that the far field approximation is valid. 

In [11]:
for loop in np.arange(numFrames):
    # get object patch
    row, col = positions[loop]
    sy = slice(row, row + Np)
    sx = slice(col, col + Np)
    # note that object patch has size of probe array
    objectPatch = obj[..., sy, sx].copy()
    # multiply each probe mode with object patch
    esw = objectPatch*probe
    # generate diffraction data, propagate the esw to the detector plane
    ESW = np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(esw), norm='ortho'))
    # save data in ptychogram
    ptychogram[loop] = abs(ESW)**2

Next the ptychogram is plotted using a ptyLab plotting function.

In [12]:
# Show the calculated diffraction data
show3Dslider(np.log(ptychogram))

## 6. Reconstruction of the simulated data

Finally, the object is reconstructed from the generated diffraction data. For this purpose, a ptyLab reconstruction script was prepared. The script starts the reconstruction process and therefore retrieves the object and probe. The structure and functionality of the reconstruction script will be explained in the next tutorial.   

In [13]:
import matplotlib
matplotlib.use('qt5Agg')

import fracPy
from fracPy.FixedData.DefaultExperimentalData import ExperimentalData
from fracPy.Optimizables.Optimizable import Optimizable
from fracPy.Monitors.Monitor import Monitor
from fracPy.Params.ReconstructionParameters import Reconstruction_parameters
from fracPy import Engines

# optimizable, exampleData, params, monitor, ePIE_engine = fracPy.easyInitialize(filePath)

exampleData = ExperimentalData()
exampleData.ptychogram = ptychogram
exampleData.dxd = dxd
exampleData.Nd = Nd
exampleData.wavelength = wavelength
exampleData.spectralDensity = None
exampleData.zled = None
exampleData.zo = zo
exampleData.operationMode = 'CPM'
exampleData.dxp = dxp
exampleData.No = No
exampleData.entrancePupilDiameter = None
exampleData._setData()
exampleData.encoder = encoder

optimizable = Optimizable(exampleData)
optimizable.npsm = 1 # Number of probe modes to reconstruct
optimizable.nosm = 1 # Number of object modes to reconstruct
optimizable.nlambda = 1 # len(exampleData.spectralDensity) # Number of wavelength
optimizable.nslice = 1 # Number of object slice

optimizable.initialProbe = 'circ'
exampleData.entrancePupilDiameter = optimizable.Np / 3 * optimizable.dxp  # initial estimate of beam
optimizable.initialObject = 'ones'

# initialize probe and object and related params
optimizable.prepare_reconstruction()

# customize initial probe quadratic phase
optimizable.probe = optimizable.probe*np.exp(1.j*2*np.pi/optimizable.wavelength *
                                             (optimizable.Xp**2+optimizable.Yp**2)/(2*6e-3))

monitor = Monitor()
monitor.figureUpdateFrequency = 1
monitor.objectPlot = 'complex'  # complex abs angle
monitor.verboseLevel = 'high'  # high: plot two figures, low: plot only one figure
monitor.objectPlotZoom = 1.5   # control object plot FoV
monitor.probePlotZoom = 0.5   # control probe plot FoV

params = Reconstruction_parameters()
## main parameters
params.positionOrder = 'random'  # 'sequential' or 'random'
params.propagator = 'Fresnel'  # Fraunhofer Fresnel ASP scaledASP polychromeASP scaledPolychromeASP


## how do we want to reconstruct?
params.gpuSwitch = False
params.probePowerCorrectionSwitch = True
params.modulusEnforcedProbeSwitch = False
params.comStabilizationSwitch = True
params.orthogonalizationSwitch = False
params.orthogonalizationFrequency = 10
params.fftshiftSwitch = False
params.intensityConstraint = 'standard'  # standard fluctuation exponential poission
params.absorbingProbeBoundary = False
params.objectContrastSwitch = False
params.absObjectSwitch = False
params.backgroundModeSwitch = False
params.couplingSwitch = True
params.couplingAleph = 1
params.positionCorrectionSwitch = False

# mPIE
engine_mPIE = Engines.mPIE(optimizable, exampleData, params, monitor)
engine_mPIE.numIterations = 50
engine_mPIE.betaProbe = 0.25
engine_mPIE.betaObject = 0.25
engine_mPIE.reconstruct()

plt.close("all")


d:\git\fracpy\fracPy\Engines\BaseReconstructor.py:293: UserWarning: fftshiftSwitch set to false, this may lead to reduced performance
  warnings.warn('fftshiftSwitch set to false, this may lead to reduced performance')


                                                                                                                       
iteration: 0                                                                                                           
error: 68.0                                                                                                            
estimated linear overlap: 67.7 %                                                                                       
estimated area overlap: 86.0 %                                                                                         
                                                                                                                       
iteration: 1                                                                                                           
error: 39.3                                                                                                            
estimated linear overlap: 71.0 %        

estimated linear overlap: 72.4 %                                                                                       
estimated area overlap: 91.3 %                                                                                         
                                                                                                                       
iteration: 14                                                                                                          
error: 3.7                                                                                                             
estimated linear overlap: 72.4 %                                                                                       
estimated area overlap: 91.1 %                                                                                         
                                                                                                                       
iteration: 15                           

iteration: 27                                                                                                          
error: 1.9                                                                                                             
estimated linear overlap: 72.5 %                                                                                       
estimated area overlap: 91.0 %                                                                                         
                                                                                                                       
iteration: 28                                                                                                          
error: 1.8                                                                                                             
estimated linear overlap: 72.5 %                                                                                       
estimated area overlap: 91.1 %          

estimated area overlap: 91.0 %                                                                                         
                                                                                                                       
iteration: 41                                                                                                          
error: 1.5                                                                                                             
estimated linear overlap: 72.6 %                                                                                       
estimated area overlap: 90.9 %                                                                                         
                                                                                                                       
iteration: 42                                                                                                          
error: 1.5                              